In [4]:
import pickle

def load_and_print_intent_stats(bin_path):
    # Load the pickle file
    with open(bin_path, "rb") as f:
        stats = pickle.load(f)
    
    print(stats['Suryakumar Yadav'])
    # Print each batter and their metrics
    # for batter, metrics in stats.items():
    #     print(f"=== {batter} ===")
    #     for metric, value in metrics.items():
    #         print(f"{metric.replace('_', ' ').title()}: {value}")
    #     print()

if __name__ == "__main__":
    # adjust the path if needed
    paths = ["data/t20_intent_reliability_stats.bin","data/batter_intent_stats.bin","data/t20_batter_stats_360.bin","data/t20_entropy_focus.bin"]
    for p in paths:
        load_and_print_intent_stats(p)
    


{'pace_intent': 1.35, 'pace_reliability': 1.15, 'spin_intent': 1.17, 'spin_reliability': 1.16, 'pace_int_rel': 1.55, 'spin_int_rel': 1.36, 'avg_intent': 1.26, 'avg_reliability': 1.15, 'avg_int_rel': 1.45}
{'final_intent_impact': 23.363086565631995, 'impact_acceleration': 0.4767976850128979, 'negative_impact_duration': 1, 'impact_improvement': 0.179952319376109, 'improvement_rate': 0.0036724963137981428, 'improvement_pct': 54.166666666666664}
{'score360': np.float64(514.53), 'ground_coverage_pct': np.float64(71.1), 'audacity': np.float64(5.49), 'aggressive_bp_pct': 33.6, 'overall_bp_pct': 26.5, 'diff_shot_effect': 1.27}
{'primary_focus': 'Length', 'least_affected_by': 'Field Changes'}
